In [1]:
import os
import random
import shutil
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from PIL import Image
import scipy
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

2023-11-30 10:14:48.625347: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 10:14:48.653718: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-30 10:14:48.653743: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-30 10:14:48.654543: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-30 10:14:48.659322: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 10:14:48.659610: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
# Define paths
main_folder = '/home/shuruyi/Documents/qgis/google_satelite/extracted_images/'
train = '/home/shuruyi/Documents/qgis/google_satelite/train'
test = '/home/shuruyi/Documents/qgis/google_satelite/test'

# Create train and test directories if they don't exist
os.makedirs(train, exist_ok=True)
os.makedirs(test, exist_ok=True)

# List all TIFF files in the main folder
tif_files = [f for f in os.listdir(main_folder) if f.endswith('.tif')]

# Define the ratio for train-test split (e.g., 80-20 split)
train_ratio = 0.8
test_ratio = 1 - train_ratio

# Randomly shuffle the list of files
random.shuffle(tif_files)

# Create lists to store filenames and labels for files moved to the test folder
test_files = []
test_labels = []
train_files = []

# Move files to train and test folders based on the ratios
for file in tif_files:
    src = os.path.join(main_folder, file)

    # Extract label from the filename (assuming the label is embedded in the filename)
    label = int(file.split('_label_')[-1].split('.')[0])

    # Store filename and label for test dataset
    if len(test_files) < len(tif_files) * test_ratio:
        test_files.append(file)
        test_labels.append(label)
        dst = os.path.join(test, file)
    else:
        train_files.append(file)
        # Define the destination folder based on the label
        if label == 0:
            dst = os.path.join(train, 'not-pinus', file)
        else:
            dst = os.path.join(train, 'pinus', file)

    # Move the file to the appropriate subfolder
    shutil.move(src, dst)

# Create a DataFrame with filenames and labels for files moved to the test folder
test_data = {'filename': test_files, 'label': test_labels}
test_df = pd.DataFrame(test_data)

# Save the DataFrame as a CSV file for the test data
csv_filename = '/home/shuruyi/Documents/qgis/google_satelite/test_labels.csv'
test_df.to_csv(csv_filename, index=False)

# Create a DataFrame with filenames for files moved to the train folder
train_data = {'filename': train_files}
train_df = pd.DataFrame(train_data)

In [6]:
train_df

,filename
0,extracted_roi_884_label_1.tif
1,extracted_roi_175_label_1.tif
2,extracted_roi_715_label_0.tif
3,extracted_roi_481_label_1.tif
4,extracted_roi_745_label_1.tif
...,...
1742,extracted_roi_848_label_0.tif
1743,extracted_roi_886_label_1.tif
1744,extracted_roi_1110_label_1.tif
1745,extracted_roi_1091_label_1.tif


In [7]:
# CNN model parameters
img_width, img_height = 150, 150
input_shape = (img_width, img_height, 3)
epochs = 10
batch_size = 32

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

# Define CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=epochs)

# Get list of all .tif files in the test folder
test_image_files = [os.path.join(test, f) for f in os.listdir(test) if f.endswith('.tif')]

# Prepare data for testing
test_images = []
for image_path in test_image_files:
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_width, img_height))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img / 255.0  # Normalize pixel values
    test_images.append(img)

# Convert the list of images to numpy array
test_images = np.array(test_images)

# Get predictions for the test data
predictions = model.predict(test_images)

Found 1747 images belonging to 2 classes.


Epoch 1/10
54/54 [==============================] - 154s 3s/step - loss: 0.6333 - accuracy: 0.6478
Epoch 2/10
54/54 [==============================] - 141s 3s/step - loss: 0.4659 - accuracy: 0.7878
Epoch 3/10
54/54 [==============================] - 134s 2s/step - loss: 0.3759 - accuracy: 0.8379
Epoch 4/10
54/54 [==============================] - 133s 2s/step - loss: 0.3351 - accuracy: 0.8612
Epoch 5/10
54/54 [==============================] - 160s 3s/step - loss: 0.2952 - accuracy: 0.8840
Epoch 6/10
54/54 [==============================] - 165s 3s/step - loss: 0.3534 - accuracy: 0.8583
Epoch 7/10
54/54 [==============================] - 190s 4s/step - loss: 0.3083 - accuracy: 0.8764
Epoch 8/10
54/54 [==============================] - 178s 3s/step - loss: 0.2445 - accuracy: 0.9044
Epoch 9/10
54/54 [==============================] - 171s 3s/step - loss: 0.2112 - accuracy: 0.9201
Epoch 10/10
14/14 [==============================] - 1s 82ms/step


In [8]:
# Define the file path where you want to save the CSV file
csv_filename = '/home/shuruyi/Documents/qgis/google_satelite/test_labels.csv'

# Save the true_pred DataFrame to a CSV file
true_pred = pd.read_csv(csv_filename)

# Convert probabilities to binary predictions (0 or 1) using a threshold (e.g., 0.5)
binary_predictions = (predictions > 0.5).astype(int)

# Create a DataFrame for binary predictions
binary_predictions_df = pd.DataFrame(binary_predictions, columns=['Binary_Predictions'])

# Concatenate train_df with binary_predictions_df
concatenated_df = pd.concat([test_df, binary_predictions_df], axis=1)

# Print the binary predictions
concatenated_df = concatenated_df.drop("label", axis=1)
concatenated_df

,filename,Binary_Predictions
0,extracted_roi_631_label_1.tif,1
1,extracted_roi_857_label_1.tif,1
2,extracted_roi_602_label_0.tif,1
3,extracted_roi_1228_label_1.tif,1
4,extracted_roi_255_label_1.tif,1
...,...,...
432,extracted_roi_104_label_1.tif,1
433,extracted_roi_182_label_0.tif,1
434,extracted_roi_528_label_0.tif,0
435,extracted_roi_195_label_0.tif,0


In [9]:
# Assuming true labels are stored in a variable called true_labels
true_labels = true_pred['label'].values  # Extracting true labels from the DataFrame

# Assuming predicted binary values are in the concatenated_df DataFrame under 'Binary_Predictions' column
predicted_values = concatenated_df['Binary_Predictions'].values  # Extracting predicted binary values

# Calculate accuracy using accuracy_score
accuracy = accuracy_score(true_labels, predicted_values)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 49.89%
